In [10]:
import numpy as np
import pandas as pd 
from functools import reduce
from operator import itemgetter
from userCF import UserCF
class ItemCF(UserCF):
	"""docstring for ItemCF apply to dataset of moivelens"""
	def __init__(self,filePath=None,k=5):
		super(ItemCF, self).__init__(filePath=filePath,k=k)

	def itmeSimilarity(self):
		user_item = self.train
		item_user = self._reverse_iu(self.train)
		w = {u:{} for u in item_user.keys()}

		log_dict = {}
		for user,item in user_item.items():
			if user in log_dict:
				log_dict[user]+= 1/np.log(1+len(item))
			else:
				log_dict[user] = 1/np.log(1+len(item))

		for user,items in user_item.items():
			for itemi in items:
				for itemj in items:
					if itemi==itemj:
						continue
					else:
						if itemj in w[itemi]:
							w[itemi][itemj]+=log_dict[user]
						else:
							w[itemi][itemj]=log_dict[user]

		for itemi in item_user.keys():
			for itemj in w[itemi].keys():
				w[itemi][itemj] /= np.sqrt(len(item_user[itemi])*len(item_user[itemj]))
		for itemi in item_user.keys(): #归一化
			maxs = 0
			for pi in w[itemi].values():
				maxs = max(maxs,pi)
			for itemj in w[itemi].keys():
				w[itemi][itemj] /= maxs


		for itemu , dicts_item in w.items():
			tuples = sorted(dicts_item.items(),key = itemgetter(1),reverse=True)[:self.k]
			w[itemu] = {i:pi for i,pi in tuples}
		self.w = w

	def recommend(self,commend_num=10):
		rank={}
		for user in self.test.keys():
			local_rank = {}
			favorite_items = self.train[user]

			for item in favorite_items:
				for it,pi in self.w[item].items():
					if it in self.train[user]:
						continue
					else:
						if it in local_rank:
							local_rank[it] += pi
						else:
							local_rank[it] = pi 
			rank[user] = local_rank
		self.rank = rank
		return self._pick(commend_num)

	def _pick(self,commend_num):
		prediction = {}
		for user,ranks in self.rank.items():
			tuples = sorted(ranks.items(),key = itemgetter(1),reverse = True)
			prediction[user]=[u for u,_ in tuples[:commend_num]]
		self.prediction = prediction
		return self.prediction

	def _reverse_iu(self,dicts):
		reverse_out = {}
		for before,after in dicts.items():
			for af in after:
				if af not in reverse_out:
					reverse_out[af] = [before]
				else:
					reverse_out[af].append(before)
		return reverse_out




In [11]:
filePath = r"ml-1m\moiveLens.csv"
itemCF = ItemCF(filePath=filePath,k=10)
itemCF.split_data(0)
itemCF.itmeSimilarity()
prediction = itemCF.recommend()

successfully load the file
suger:


In [12]:
itemCF.precision(prediction)

0.23768961493582264

In [13]:
itemCF.recall(prediction)

0.1140482779581847

In [14]:
itemCF.coverage(prediction)

0.22303523035230352

In [15]:
itemCF.popularity(prediction)

7.24248077985019